In [2]:
import matplotlib.pyplot as plt
import scipy.linalg as sl
import autograd.numpy.linalg as la
import autograd.numpy as np
import sys
import os
from datetime import datetime
from matplotlib.collections import LineCollection

n_d = 22
NB = 83
netpath = '/Users/jiayiwu/experiment/bio/bio_network_2'
name = "/network_tri_num_p_%d_num_b_%d_fixed_" % (n_d, NB)
C = np.load(netpath + name + "Cde.npy")
points = np.load(netpath + name + "points.npy")
i_points = np.load(netpath + name + "i_points.npy")
connection = np.load(netpath + name + "Connection.npy")

# tri = Delaunay(points)
NB, dof = np.shape(C)
print(dof)

nums = [1, 2, 19, 20]

v1 = np.zeros(dof)

v1[nums[0] * 2:nums[0] * 2 + 4] = [1, 0, -1, 0]
v1[nums[2] * 2:nums[2] * 2 + 4] = [-1, 0, 1, 0]

indT1 = np.where(v1 > 0)[0]
indF1 = np.where(v1 < 0)[0]
zos1 = [nums[0] * 2 + 1, nums[0] * 2 + 3, nums[2] * 2 + 1, nums[2] * 2 + 3]

v1 = v1 / la.norm(v1)

v2 = np.zeros(dof)

v2[nums[0] * 2:nums[0] * 2 + 4] = [1, 0, -1, 0]
v2[nums[2] * 2:nums[2] * 2 + 4] = [1, 0, -1, 0]

indT2 = np.where(v2 > 0)[0]
indF2 = np.where(v2 < 0)[0]

v2 = v2 / la.norm(v2)


44


In [3]:
def ENE(fev1, indT, indF, zos):
    return thes * 4 - lthes * 4 + sum(-np.minimum(fev1[indT], thes)) + sum(np.maximum(fev1[indF], -thes)) + sum(
        np.maximum(abs(fev1[zos]), lthes))


def E12(k, sv):
    D = D_from_k(k, C)  # get the dynamic matrix
    e1, ev1 = sl.eigh(D)  # np.linalg.eig(D)  # get the eigval and eigvect

    seq = np.argsort(e1)  # sort the eigval

    if sv == 'v1':
        indT = indT1
        indF = indF1
    if sv == 'v2':
        indT = indT2
        indF = indF2

    zos = zos1

    return min(ENE(ev1[:, seq[0]], indT, indF, zos), ENE(-ev1[:, seq[0]], indT, indF, zos))


def fdiff_grad_eigvec(k, indT, indF, zos, w2=1.):
    D = D_from_k(k, C)  # get the dynamic matrix
    e1, ev1 = sl.eigh(D)  # get the eigval and eigvect

    seq = np.argsort(e1)  # sort the eigval

    fvec1 = min(ENE(ev1[:, seq[0]], indT, indF, zos), ENE(-ev1[:, seq[0]], indT, indF, zos))
    n1, n2 = np.sort(e1)[:2]
    dis1 = -np.arctan((n2 - n1) / gamma)

    dk1 = np.zeros(len(k))
    dk2 = np.zeros(len(k))

    dk = np.zeros(len(k))
    eps = 0.00001

    for i in range(len(k)):
        ki = k.copy()
        ki[i] += eps
        D2 = D_from_k(ki, C)
        e2, ev2 = sl.eigh(D2)

        seq = np.argsort(e2)

        fvec2 = min(ENE(ev2[:, seq[0]], indT, indF, zos), ENE(-ev2[:, seq[0]], indT, indF,
                                                              zos))
        dk[i] = (fvec2 - fvec1) / eps

        n1i, n2i = np.sort(e2)[:2]

        dk1[i] = (n1i - n1) / eps
        dk2[i] = (n2i - n2) / eps

    grad = dk - w2 * (1. / (((n2 - n1) / gamma) ** 2 + 1) * (dk2 - dk1))  # + eta * np.exp(-(k0 +1) * eta) * dk1s)
    cons_ene = dis1  # - eta * np.exp(-eta * (k0+1))

    return grad, fvec1, cons_ene


def grad_des(run_time, eps, k, indT, indF, zos, w2=1):
    energy = np.zeros(run_time)

    precision = 10 ** (-10)
    cons_ene = np.zeros(run_time)
    E1 = []
    for i in range(run_time):

        grad1, ene, cons = fdiff_grad_eigvec(k, indT, indF, zos, w2)
        # grad2 = grad(auto_energy)(k)
        gradient = grad1  # +grad2
        # gradient[abs(gradient)<np.argmax(abs(gradient))] =0

        gradient[gradient > 10] = 0

        gradient[gradient == np.nan] = 0
        # aa = np.where(gradient == max(gradient))

        k = k - eps * gradient
        # k[aa] = 0
        k[k < 0] = 0

        energy[i] = ene
        cons_ene[i] = cons
        E1.append([E12(k, 'v1'), E12(k, 'v2')])

        if i > 1 and la.norm(gradient) < precision:
            print('converaged at step=%d with ene %0.2f ' % (i, energy[i]))
            return k, np.array([energy[:i], cons_ene[:i]]), E1

    return k, np.array([energy, cons_ene]), E1


def D_from_k(k, A):
    return A.T @ np.diag(k) @ A


def plot_eig(k, v, fv, sep_datapath, name, indT, indF, ylim=1):
    D = D_from_k(k, C)

    eigval, eigvect = sl.eigh(D)
    seq = np.argsort((eigval))
    print('eigval are', np.round(eigval[seq], 3))
    plt.figure(figsize=(10, 6))

    plt.subplot(121)
    plt.plot(v, 'c--', alpha=0.8, linewidth=2, label='target')
    plt.scatter(range(0, len(v)), v[:len(v)])

    if ENE(eigvect[:, seq[0]], indT, indF, zos1) < ENE(-eigvect[:, seq[0]], indT, indF, zos1):
        fvec1 = eigvect[:, seq[0]]
    else:
        fvec1 = -eigvect[:, seq[0]]

    plt.plot(fvec1, 'r--')
    plt.scatter(fv, fvec1[fv], color='r')
    plt.title(str(name)[-2:] + ' at cycle: ' + str(name)[:len(str(name)) - 2], fontsize=20)
    plt.ylim(-0.4, 0.4)

    eig = np.sort(eigval)

    ax1 = plt.subplot(133)
    aa = np.reshape(np.repeat(eig, 2), (-1, 2))
    # print(aa)
    for i in range(10):
        plt.plot(aa[i], label=str(np.round(eig[i], 2)))

    plt.ylim(min(eigval) - 0.1, ylim + min(eigval) - 0.1)
    x_axis = ax1.axes.get_xaxis()
    x_axis.set_visible(False)
    plt.legend(bbox_to_anchor=(-0.6, 1.), loc=2, borderaxespad=0.)

    plt.savefig(sep_datapath + '/' + str(name) + '.png')
    # plt.show()
    plt.close()


def multiline(xs, ys, c, ax=None, **kwargs):
    """Plot lines with different colorings

    Parameters
    ----------
    xs : iterable container of x coordinates
    ys : iterable container of y coordinates
    c : iterable container of numbers mapped to colormap
    ax (optional): Axes to plot on.
    kwargs (optional): passed to LineCollection

    Notes:
        len(xs) == len(ys) == len(c) is the number of line segments
        len(xs[i]) == len(ys[i]) is the number of points for each line (indexed by i)

    Returns
    -------
    lc : LineCollection instance.
    """

    # find axes
    ax = plt.gca() if ax is None else ax

    # create LineCollection
    segments = [np.column_stack([x, y]) for x, y in zip(xs, ys)]
    lc = LineCollection(segments, **kwargs)

    # set coloring of line segments
    #    Note: I get an error if I pass c as a list here... not sure why.
    lc.set_array(np.asarray(c))

    # add lines to axes and rescale
    #    Note: adding a collection doesn't autoscalee xlim/ylim
    ax.add_collection(lc)
    ax.autoscale()
    return lc

def get_lines(xx,yy,NB=NB):
    xcon = []
    ycon = []
    for i in range(NB):

        if int(connection[i][0]) < len(xx):
            x0 = xx[int(connection[i][0])]
            y0 = yy[int(connection[i][0])]
        else:
            x0, y0 = points[int(connection[i][0])]
        if int(connection[i][1]) < len(xx):
            x1 = xx[int(connection[i][1])]
            y1 = yy[int(connection[i][1])]
        else:
            x1, y1 = points[int(connection[i][1])]

        xcon.append([x0, x1])
        ycon.append([y0, y1])

    xcon = np.array(xcon)
    ycon = np.array(ycon)
    return xcon, ycon
def plot_field(xx, yy,ip = i_points):
    plt.scatter(xx, yy, color='r', alpha=1)
    for i in range(len(ip)):
        if i in nums:
            alp = 0.9
            c = 'blue'
            hw = 12

        else:
            c = 'black'
            alp = 0.3
            hw = 8

        plt.annotate("", xytext=(ip[:, 0][i], ip[:, 1][i]), xy=(xx[i], yy[i]),
                     arrowprops=dict(width=1.5, headwidth=hw, color=c, alpha=alp))

   
    
def plot_act_net(points, ip, k, dirp, name, indT, indF, v, amp=1):
    long = (int(max(points[:, 0]) - min(points[:, 0]) + 4)) % 20
    wid = (int(max(points[:, 1]) - min(points[:, 1]) + 4)) % 20
    print(long, wid)
    fig = plt.figure(figsize=(long, wid))

    plt.plot(points[:, 0], points[:, 1], 'o', color='black', alpha=0.2)
    plt.plot(ip[:, 0], ip[:, 1], 'o', color='black', alpha=0.2)
    D = D_from_k(k, C)

    eigval, eigvect = sl.eigh(D)
    seq = np.argsort((eigval))
    xs = range(0, dof, 2)
    ys = range(1, dof + 1, 2)

    if ENE(eigvect[:, seq[0]], indT, indF, zos1) > ENE(-eigvect[:, seq[0]], indT, indF, zos1):
        fev1 = -eigvect[:, seq[0]]
        xx = -amp * eigvect[:, seq[0]][xs] + ip[:, 0]
        yy = -amp * eigvect[:, seq[0]][ys] + ip[:, 1]
    else:
        fev1 = eigvect[:, seq[0]]
        xx = amp * eigvect[:, seq[0]][xs] + ip[:, 0]
        yy = amp * eigvect[:, seq[0]][ys] + ip[:, 1]
    
    plot_field(xx, yy)
    for i in np.concatenate((indT, indF)):

        xi = points[:, 0][int(i / 2)]
        yi = points[:, 1][int(i / 2)]
        if yi > 1:
            jj = 0.5
        else:
            jj = -0.5
        plt.annotate("", xy=(xi + v[i], yi + jj), xytext=(xi, yi + jj),
                     arrowprops=dict(width=1.5, color='black', alpha=0.8))

    
    plt.axes().set_aspect('equal', 'datalim')
    xcon,ycon = get_lines(xx,yy)

    elongation = C @ fev1
    lc = multiline(xcon, ycon, elongation, cmap='plasma', lw=2, alpha=0.7)
    axcb = fig.colorbar(lc)
    axcb.set_label('bond elongation ')

    plt.ylim(min(points[:, 1]) - 0.5, max(points[:, 1]) + 0.5)
    plt.xlim(min(points[:, 0]) - 0.5, max(points[:, 0]) + 0.5)
    plt.axis('off')
    plt.title(str(name)[-2:] + ' at: ' + str(name)[:len(str(name)) - 2] +
              ' eigval = %0.2f with amplify :%d' % (eigval[seq[0]], amp), fontsize=20)
    plt.savefig(dirp + '/network_' + str(name) + '.pdf')
    print(eigval[seq][:5])
    plt.close()
    # plt.show()
    
    fig = plt.figure(figsize=(long, wid))
    plot_field(xx, yy)
    xcon,ycon = get_lines(ip[:,0],ip[:,1])
    lc = multiline(xcon, ycon, k, cmap='coolwarm', lw=2, alpha=1)
    axcb = fig.colorbar(lc)
    axcb.set_label('k')
    plt.axes().set_aspect('equal', 'datalim')

    plt.ylim(min(points[:, 1]) - 0.5, max(points[:, 1]) + 0.5)
    plt.xlim(min(points[:, 0]) - 0.5, max(points[:, 0]) + 0.5)
    plt.axis('off')
    plt.title(str(name)[-2:] + ' at: ' + str(name)[:len(str(name)) - 2] +
              ' eigval = %0.2f with amplify :%d' % (eigval[seq[0]], amp), fontsize=20)
    plt.savefig(dirp + '/k_network_' + str(name) + '.pdf')

    plt.close()

    # ake another one for k
    


def osc_goals(k, it, T, plotit=True, j=0):
    dif_ene = []
    dif_bond = []
    E1 = []
    plt.clf()
    plt.close()
    c_t_1 = []
    c_t_2 = []
    i = 0

    while i < T:

        k, energy1, E11 = grad_des(it, eps, k, indT2, indF2, zos1)
        plot_eig(k, v2, np.where(v2 != 0)[0], opath, '%02dv2' % i, indT2, indF2)
        plot_act_net(points, i_points, k, opath, '%02dv2' % i, indT2, indF2, v2)
        print(i)

        prev_k = k.copy()

        Ev2 = np.array(E11).T[1]
        E1 += E11

        if Ev2[-1] < precision:
            c_t_2 += [np.amin(np.where(Ev2 < precision)) / it]

        else:
            c_t_2 += [1]
        # print('end ene for v2 is', Ev2[-1])

        k, energy1, E11 = grad_des(it, eps, k, indT1, indF1, zos1)
        E1 += E11

        Ev1 = np.array(E11).T[0]
        # print('end ene for v1 is', Ev1[-1])

        if Ev1[-1] < precision:
            c_t_1 += [np.amin(np.where(Ev1 < precision)) / it]

        else:
            c_t_1 += [1]

        dif_ene += [la.norm(prev_k - k, 1)]
        diff = prev_k - k
        # print(diff)

        diff[abs(diff) < 0.5 * k] = 0

        dif_bond += [np.count_nonzero(diff)]
        plot_eig(k, v1, np.where(v1 != 0)[0], opath, '%02dv1' % i, indT1, indF1)
        plot_act_net(points, i_points, k, opath, '%02dv1' % i, indT1, indF1, v1)

        if c_t_1[-1] != 1 and c_t_2[-1] != 1:
            flag = True
            print('osc coveraged at', i)
            break
        

        

        i += 1

    if c_t_1[-1] == 1 or c_t_2[-1] == 1:

        flag = False
        print('osc didnt coverage')


        # return dif_bond, dif_ene, flag
    else:
        bondnum = np.where(abs(diff) > 0.5 * k)
        flag = True
        np.save(tpath + '/%02dv1_%d' % (j, T), prev_k)
        np.save(tpath + '/%02dv2_%d' % (j, T), k)
        np.save(tpath + '/%02d_bond_nums' % j, bondnum)

    if plotit:
        plt.clf()
        plt.close()
        plt.plot(c_t_1, label='v1 converage time')
        plt.plot(c_t_2, label='v2 converage time')
        plt.legend()
        # plt.xlim(0,20)
        plt.ylabel('converage time', fontsize=20)
        plt.xlabel('cycles', fontsize=20)
        plt.savefig(dirpath + '/T_%d_rt_%d_converage_time_%04d.png' % (T, it, j), dpi=100,
                    bbox_inches='tight')
        plt.close()
        # from mpl_toolkits.axes_grid.inset_locator import inset_axes

        fig = plt.figure(figsize=(7, 6))
        ax = fig.add_subplot(111)
        Ear = np.array(E1)

        # nn = 1  # np.max([int(it / 100), 1])

        ax.plot(np.arange(len(Ear.T[0])), Ear.T[0], label='G1')  # , s = 0.01)
        ax.plot(np.arange(len(Ear.T[1])), Ear.T[1], label='G2')
        # plt.plot(range(len(Ear.T[2])),Ear.T[2], label = 'E3 (Av3) changing over time')
        ax.set_ylabel('Energy changing over time')
        ax.set_xlabel('real time')
        fig.legend(bbox_to_anchor=(0.5, 0.55), loc=2, borderaxespad=0.)

        plt.ylim(ymin=0)

        plt.savefig(dirpath + '/T_%04d_rt_%d_Ene_A1_%04d.pdf' % (T, it, j), dpi=100)
        plt.close()
        # if flag:
        #    plot_eig(k, v1, np.where(v1 != 0)[0], tpath, '%02dv1' % j, indT1, indF1)
        #    plot_act_net(points, i_points, k, tpath, '%02dv1' % j, indT1, indF1, v1)

    #             plot_eig(prev_k, v2, np.where(v2 != 0)[0], tpath, '%02dv2' % j, indT2, indF2)
    #             plot_act_net(points, i_points, prev_k, tpath, '%02dv2' % j, indT2, indF2, v2)

    return dif_ene, dif_bond, flag

In [4]:
Kall = np.load('/Users/jiayiwu/experiment/bio/bio_mec_s/ks_it_1000_500.npy')

In [8]:
thes = 0.1
lthes = 0.05

gamma = 0.05  # making n2-n1 big
# eta = 0.5
precision = 0.05
eps = 0.1

run_time = 10000
it = int(run_time/T)
ori_k = Kall[i]




dif_ene_3, dif_bond, flag = osc_goals(ori_k, it, T, True, i)


eigval are [0.094 0.174 0.212 0.278 0.311 0.349 0.41  0.484 0.587 0.646 0.676 0.706
 0.809 0.817 0.861 0.905 0.948 1.052 1.12  1.138 1.183 1.299 1.315 1.489
 1.528 1.534 1.573 1.839 1.872 1.974 2.043 2.156 2.168 2.262 2.384 2.45
 2.57  2.721 2.88  3.19  3.325 3.406 3.568 3.716]
10 9


/Users/jiayiwu/anaconda/envs/bio/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


[0.09426758 0.17403453 0.21237012 0.27769688 0.31058416]
0
eigval are [0.073 0.175 0.208 0.275 0.3   0.337 0.445 0.504 0.601 0.655 0.674 0.708
 0.805 0.825 0.847 0.885 0.951 1.049 1.125 1.135 1.187 1.314 1.373 1.493
 1.538 1.544 1.587 1.84  1.887 1.987 2.058 2.18  2.21  2.275 2.389 2.445
 2.584 2.739 2.89  3.19  3.296 3.407 3.583 3.68 ]
10 9
[0.07252595 0.1745686  0.2083266  0.27518286 0.30011899]
eigval are [0.061 0.169 0.21  0.276 0.302 0.322 0.428 0.483 0.599 0.645 0.658 0.713
 0.807 0.818 0.846 0.896 0.956 1.059 1.134 1.136 1.2   1.321 1.379 1.502
 1.535 1.55  1.587 1.841 1.887 2.003 2.07  2.192 2.226 2.285 2.397 2.438
 2.607 2.751 2.91  3.2   3.308 3.408 3.601 3.686]
10 9
[0.06072365 0.1690226  0.20969991 0.27632126 0.30182655]
1
eigval are [0.079 0.194 0.208 0.285 0.307 0.359 0.457 0.516 0.615 0.648 0.668 0.713
 0.76  0.831 0.857 0.911 0.96  1.069 1.13  1.143 1.214 1.331 1.421 1.511
 1.541 1.566 1.604 1.827 1.901 2.008 2.079 2.205 2.259 2.304 2.404 2.453
 2.616 2.771 2.925 3.183 

[0.1551991  0.26707594 0.2787743  0.33643357 0.37902635]
12
eigval are [0.156 0.272 0.273 0.339 0.393 0.484 0.522 0.586 0.612 0.687 0.727 0.765
 0.827 0.869 0.976 1.019 1.054 1.143 1.264 1.283 1.328 1.373 1.475 1.585
 1.705 1.797 1.845 1.952 2.092 2.259 2.379 2.428 2.548 2.605 2.73  2.799
 2.937 3.017 3.177 3.249 3.499 3.648 3.914 4.203]
10 9
[0.15646946 0.27186444 0.27332424 0.33938528 0.39326179]
eigval are [0.153 0.278 0.281 0.352 0.387 0.494 0.516 0.597 0.627 0.714 0.755 0.781
 0.832 0.891 0.97  1.021 1.028 1.157 1.272 1.302 1.329 1.378 1.473 1.594
 1.718 1.815 1.854 1.962 2.109 2.252 2.393 2.433 2.563 2.604 2.74  2.808
 2.941 3.021 3.155 3.254 3.499 3.646 3.931 4.232]
10 9
[0.15278419 0.27752983 0.28098369 0.35197678 0.38727431]
13
eigval are [0.152 0.274 0.276 0.343 0.4   0.487 0.518 0.589 0.614 0.69  0.731 0.768
 0.829 0.875 0.987 1.029 1.043 1.15  1.275 1.293 1.336 1.38  1.477 1.597
 1.716 1.81  1.857 1.966 2.106 2.274 2.398 2.439 2.564 2.619 2.742 2.808
 2.959 3.038 3.199 3.26

[0.14309946 0.30190876 0.30425282 0.38377337 0.43090939]
24
eigval are [0.144 0.3   0.302 0.379 0.441 0.502 0.529 0.619 0.649 0.698 0.775 0.788
 0.863 0.89  1.022 1.036 1.126 1.207 1.346 1.364 1.422 1.45  1.518 1.671
 1.796 1.892 1.976 2.07  2.199 2.383 2.512 2.585 2.668 2.779 2.827 2.939
 3.105 3.22  3.328 3.499 3.808 3.869 4.244 4.658]
10 9
[0.14362661 0.29985876 0.30188981 0.37944141 0.44118636]
eigval are [0.144 0.306 0.307 0.39  0.435 0.507 0.543 0.649 0.679 0.75  0.766 0.82
 0.869 0.923 1.007 1.029 1.129 1.223 1.37  1.372 1.424 1.459 1.516 1.675
 1.807 1.906 1.976 2.081 2.207 2.384 2.521 2.59  2.667 2.786 2.835 2.968
 3.111 3.225 3.297 3.494 3.806 3.868 4.265 4.684]
10 9
[0.14385317 0.30556746 0.30725978 0.39045232 0.4349755 ]
25
eigval are [0.145 0.303 0.305 0.385 0.443 0.506 0.532 0.618 0.654 0.701 0.778 0.797
 0.865 0.895 1.024 1.036 1.138 1.213 1.348 1.37  1.43  1.455 1.522 1.675
 1.802 1.899 1.986 2.078 2.208 2.389 2.519 2.6   2.676 2.793 2.833 2.952
 3.116 3.236 3.335 3.519

...

10 9
[0.16833053 0.42690874 0.4277041  0.52831753 0.61046709]
144
eigval are [0.158 0.42  0.422 0.513 0.577 0.64  0.648 0.794 0.818 0.899 0.965 1.02
 1.065 1.086 1.142 1.282 1.458 1.527 1.578 1.671 1.738 1.839 1.964 2.066
 2.26  2.324 2.454 2.545 2.769 2.887 2.955 3.073 3.19  3.504 3.548 3.798
 3.882 4.112 4.505 4.782 5.306 5.527 5.863 6.872]
10 9
[0.15795154 0.4202144  0.42180098 0.51301173 0.57673859]
eigval are [0.169 0.428 0.428 0.529 0.611 0.656 0.704 0.814 0.906 0.927 0.993 1.007
 1.08  1.121 1.164 1.288 1.475 1.544 1.596 1.679 1.738 1.855 1.975 2.074
 2.272 2.355 2.445 2.544 2.784 2.885 2.962 3.077 3.192 3.515 3.576 3.799
 3.923 4.115 4.502 4.766 5.311 5.526 5.876 6.888]
10 9
[0.16856016 0.42766632 0.42838531 0.52907302 0.61114005]
145
eigval are [0.158 0.421 0.423 0.514 0.577 0.641 0.649 0.796 0.819 0.9   0.966 1.02
 1.066 1.088 1.144 1.285 1.46  1.528 1.58  1.673 1.739 1.842 1.966 2.068
 2.263 2.327 2.456 2.547 2.773 2.89  2.958 3.075 3.194 3.508 3.553 3.803
 3.888 4.118 4.513

10 9
[0.1707324  0.43531954 0.43617411 0.53609938 0.61828435]
156
eigval are [0.16  0.429 0.43  0.52  0.583 0.65  0.662 0.809 0.829 0.915 0.98  1.026
 1.082 1.105 1.161 1.304 1.477 1.545 1.596 1.692 1.755 1.866 1.988 2.096
 2.294 2.354 2.478 2.575 2.812 2.925 2.995 3.098 3.232 3.552 3.596 3.856
 3.946 4.186 4.6   4.865 5.402 5.647 5.967 7.018]
10 9
[0.16029954 0.42922885 0.4303697  0.52027813 0.58348029]
eigval are [0.171 0.435 0.438 0.535 0.618 0.667 0.714 0.825 0.919 0.942 1.007 1.011
 1.097 1.14  1.185 1.309 1.494 1.565 1.615 1.699 1.756 1.883 1.999 2.105
 2.307 2.385 2.47  2.574 2.827 2.923 3.    3.1   3.235 3.564 3.622 3.856
 3.987 4.19  4.597 4.849 5.408 5.647 5.98  7.033]
10 9
[0.1712266  0.435202   0.43765828 0.53531244 0.61753366]
157
eigval are [0.161 0.431 0.431 0.523 0.584 0.653 0.662 0.808 0.83  0.917 0.982 1.029
 1.084 1.108 1.163 1.304 1.478 1.547 1.598 1.694 1.757 1.868 1.989 2.099
 2.297 2.356 2.48  2.577 2.815 2.928 2.997 3.1   3.235 3.556 3.599 3.862
 3.951 4.192 4.6

10 9
[0.17352452 0.44233598 0.44496891 0.54146484 0.62364198]
168
eigval are [0.162 0.436 0.438 0.523 0.588 0.657 0.677 0.821 0.839 0.927 0.993 1.028
 1.097 1.122 1.179 1.325 1.494 1.561 1.615 1.712 1.772 1.891 2.01  2.126
 2.327 2.382 2.501 2.604 2.851 2.961 3.034 3.121 3.272 3.599 3.641 3.911
 4.007 4.257 4.69  4.945 5.494 5.763 6.068 7.157]
10 9
[0.1616171  0.43646039 0.43834276 0.52325741 0.58809334]
eigval are [0.174 0.443 0.445 0.542 0.624 0.677 0.726 0.835 0.933 0.956 1.011 1.026
 1.113 1.158 1.202 1.333 1.512 1.58  1.632 1.716 1.775 1.909 2.022 2.135
 2.338 2.415 2.494 2.603 2.867 2.96  3.038 3.124 3.276 3.609 3.667 3.913
 4.051 4.26  4.688 4.929 5.502 5.762 6.081 7.173]
10 9
[0.17372401 0.44315866 0.44549766 0.54236371 0.62440552]
169
eigval are [0.165 0.44  0.441 0.532 0.592 0.666 0.678 0.822 0.84  0.935 0.999 1.038
 1.103 1.128 1.18  1.327 1.495 1.563 1.616 1.714 1.774 1.893 2.011 2.128
 2.33  2.385 2.503 2.607 2.853 2.965 3.037 3.125 3.275 3.602 3.644 3.922
 4.015 4.262 4.6

10 9
[0.17577252 0.45065225 0.4523225  0.54923126 0.62992087]
180
eigval are [0.164 0.444 0.446 0.528 0.597 0.663 0.696 0.838 0.849 0.94  1.006 1.031
 1.112 1.141 1.197 1.348 1.51  1.576 1.633 1.73  1.789 1.914 2.031 2.152
 2.358 2.411 2.523 2.633 2.887 2.996 3.073 3.144 3.311 3.641 3.684 3.967
 4.069 4.323 4.776 5.021 5.585 5.874 6.166 7.294]
10 9
[0.16369957 0.44436398 0.44624257 0.52805679 0.5969727 ]
eigval are [0.176 0.451 0.454 0.548 0.632 0.686 0.737 0.845 0.945 0.971 1.012 1.041
 1.127 1.176 1.221 1.356 1.529 1.596 1.649 1.731 1.793 1.933 2.044 2.163
 2.368 2.444 2.517 2.632 2.903 2.995 3.075 3.149 3.316 3.651 3.709 3.971
 4.114 4.326 4.775 5.005 5.594 5.873 6.179 7.309]
10 9
[0.17627891 0.45067116 0.45355478 0.54847864 0.63152384]
181
eigval are [0.165 0.445 0.447 0.53  0.598 0.666 0.698 0.84  0.85  0.941 1.008 1.032
 1.114 1.141 1.198 1.351 1.511 1.577 1.634 1.731 1.79  1.917 2.033 2.155
 2.36  2.413 2.525 2.636 2.889 2.998 3.076 3.147 3.314 3.644 3.688 3.973
 4.075 4.329 4.7

10 9
[0.17821098 0.45712215 0.46015413 0.55378647 0.63772548]
192
eigval are [0.169 0.455 0.456 0.542 0.609 0.682 0.71  0.852 0.86  0.958 1.024 1.046
 1.132 1.159 1.213 1.37  1.526 1.59  1.65  1.747 1.806 1.938 2.051 2.18
 2.387 2.438 2.545 2.662 2.919 3.03  3.109 3.171 3.349 3.682 3.727 4.027
 4.13  4.388 4.862 5.095 5.679 5.981 6.261 7.424]
10 9
[0.16911604 0.45473218 0.45605402 0.54204739 0.60859382]
eigval are [0.178 0.458 0.461 0.555 0.638 0.694 0.747 0.854 0.958 0.983 1.016 1.055
 1.142 1.192 1.238 1.377 1.546 1.61  1.668 1.745 1.811 1.956 2.064 2.19
 2.397 2.472 2.539 2.661 2.936 3.029 3.108 3.174 3.356 3.691 3.751 4.026
 4.171 4.392 4.859 5.078 5.68  5.98  6.273 7.438]
10 9
[0.17779147 0.45842474 0.46082944 0.55460309 0.63840726]
193
eigval are [0.169 0.456 0.457 0.543 0.61  0.683 0.709 0.852 0.86  0.959 1.025 1.046
 1.133 1.161 1.215 1.371 1.527 1.592 1.652 1.748 1.807 1.94  2.052 2.182
 2.39  2.44  2.547 2.665 2.921 3.033 3.112 3.173 3.353 3.685 3.73  4.031
 4.135 4.394 4.868

In [7]:
T = 200
i = 2
dirpath = '/Users/jiayiwu/experiment/bio/bio_mec_2019' + '/'
opath = dirpath + '/plots%d_%d/'%(T,i)
if not os.path.exists(opath): os.mkdir(opath)
tpath = dirpath + '/switch_goal%d_%d/'%(T,i)
if not os.path.exists(tpath): os.mkdir(tpath)